An Algorithm for Solving the Factorization Problem in Permutation Groups

TORSTEN MINKWITZ

Deutsche Telekom AG, Bonn, Germany

https://www.sciencedirect.com/science/article/pii/S0747717198902024


The algorithm uses a table of coset representatives of the group stabilizers associated to a given base. The choice of this base might strongly affect the resulting word:


In [2]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import networkx as nx
from itertools import chain, product, combinations
from sympy.combinatorics.perm_groups import Permutation,PermutationGroup
from sympy.combinatorics.perm_groups import _distribute_gens_by_base,_orbits_transversals_from_bsgs

p = '../data/'
path = pd.read_csv(p + 'puzzles.csv')
info = pd.read_csv(p + 'puzzle_info.csv')
sub = pd.read_csv(p + 'sample_submission.csv')


path = pd.merge(path,info)

In [3]:
# A class for maintainig permutations and factorization over a SGS
class PermWord:
    def __init__(self, perms=[], words=[]):
        self.words = words
        self.permutation = perms
        self.flag = True

    @staticmethod
    def is_identity(self):
        return (self.words.len()==0)

    def inverse(self):
        inv_perm = ~self.permutation
        inv_word = invwords(self.words)
        return PermWord(inv_perm,inv_word)

    def __mul__(self, other):
        result_perm = self.permutation * other.permutation
        result_words = simplify(self.words + other.words)
        return PermWord(result_perm, result_words)


In [15]:
# A class generating factorization of permutations over a Strong Generating Set (SGS)
# The SGS is obtained using the sympy implementation for Schreier–Sims algorith
# The Minkwith algorithm (https://www.sciencedirect.com/science/article/pii/S0747717198902024)
# is used for mantainig a short word representation 

deterministic = False

class SGSPermutationGroup:
    def __init__(self, gensi=[]):
        gens = gensi.copy()
        gen0= [gensi[p] for p in gens]
        
        geninvs={}
        sizes = []
        for s in list(gens):
            sizes.append(gens[s].size)
            if s[0] != '-': 
                s1 = ~gens[s]
                gens["-" + s] = s1
                geninvs[s] = '-' + s
                geninvs['-' + s] = s    
        self.gens = gens
        self.geninvs = geninvs
        self.N = max(sizes)
        # Create the permutation group
        #gen0= [gens[p] for p in gens]
        G = PermutationGroup(gen0)
        self.G = G
        # obtain the strong generating set
        if deterministic:
            G.schreier_sims()
            self.base = G.base
            self.bo = G.basic_orbits
            self.bt = G.basic_transversals
            print(len(self.base))
        else:
            base,trans, orbits = schreier_sims_random(G)
            self.base = base
            self.bo = orbits
            self.bt = trans
        
     
        self.lo = [len(x) for x in self.bo]
        self.so = np.sum(self.lo)
        self.nu = None

    
    #   n: max number of rounds
    #   s: reset each s rounds
    #   w: limit for word size
    
    def getShortWords(self,n=10000,s=2000,w=20):
        self.nu = buildShortWordsSGS(self.N, self.gens, self.base, n, s, w, self.so)
        print(self.nu)


    def FactorPermutation(self,target):
        if self.nu == None:
            print('Execute getShortWords')
            return None
        return  factorizeM(self.N, self.gens, self.base, self.nu, target)

    def CheckQuality(self):
        test = test_SGS(self.N,self.nu,self.base)
        qual = quality(self.N, self.nu, self.base)
        return test,qual

    def swapBase(self,i):
        S = self.G
        base, gens = S.baseswap(S.base, S.strong_gens, i, randomized=False)
        self.base = base

        
        
def schreier_sims_random(G):
    base, strong_gens = G.schreier_sims_random(consec_succ=5)
    strong_gens_distr =_distribute_gens_by_base(base, strong_gens)
    basic_orbits, transversals, slps = _orbits_transversals_from_bsgs(base,\
                strong_gens_distr, slp=True)

    # rewrite the indices stored in slps in terms of strong_gens
    for i, slp in enumerate(slps):
        gens = strong_gens_distr[i]
        for k in slp:
            slp[k] = [strong_gens.index(gens[s]) for s in slp[k]]

    transversals = transversals
    basic_orbits = [sorted(x) for x in basic_orbits]
    return base, transversals,basic_orbits
        
def applyPerm(sol):
    if sol == []:
        return Permutation(size = PG.N)
    target = PG.gens[sol[0]]
    for m in sol[1:]:
        target = target*PG.gens[m]
    return target       


In [5]:
#Check if a solution is valid

def apply_move(moves, move, state):
    m = move
    s = state.split(';')

    move_list = moves[m]
    new_state = []
    for i in move_list:
        new_state.append(s[i])
    s = new_state

    return ';'.join(s)
    
def init_reverse_moves(moves):
    new_moves = {}
    
    for m in moves.keys():
        new_moves[m] = moves[m]
        xform = moves[m]
        m_inv = '-' + m
        xform_inv = len(xform) * [0]
        for i in range(len(xform)):
            xform_inv[xform[i]] = i
        new_moves[m_inv] = xform_inv

    return new_moves


def validate(moves, initial_state, solution_state, solution, num_wildcards):
    sols = solution.split('.')
    cur_state = initial_state
    for s in sols:
        if s not in moves:
            return False
        cur_state = apply_move(moves, s, cur_state)
    err_cnt = sum(c!=e for c,e in zip(cur_state.split(';'), solution_state.split(';')))
    if err_cnt <= num_wildcards:
        return True,cur_state
    else:
        return False,cur_state

def val_score(i,sol):
    initial_state = path.initial_state.values[i]
    solution_state = path.solution_state.values[i]
    moves = eval(path.allowed_moves.values[i])
    moves = init_reverse_moves(moves)
    num_wildcards = path.num_wildcards.values[i]
    solution = sol

    return validate(moves, initial_state, solution_state, solution, num_wildcards)

In [6]:
#

def oneStep(N, gens, base, i, t, nu):
    j = t.permutation.array_form[base[i]]  # b_i ^ t
    t1 = t.inverse()
    if nu[i][j] is not None:
        result = t * nu[i][j]
        result.words = simplify(result.words)
        if len(t.words) < len(nu[i][j].words):
            nu[i][j] = t1
            oneStep(N, gens, base, i, t1, nu)
    else:
        nu[i][j] = t1
        oneStep(N, gens, base, i, t1, nu)
        result =  PermWord(Permutation(N),[])
    return result

def oneRound(N, gens, base, lim, c, nu, t):
    i = c
    while i < len(base) and len(t.words)>0 and len(t.words) < lim:
        t = oneStep(N, gens, base, i, t, nu)
        i += 1

def oneImprove(N, gens, base, lim, nu):
    for j in range(len(base)):
        for x in nu[j]:
            for y in nu[j]:
                if x != None and y != None  and (x.flag or y.flag):
                    t = x * y
                    oneRound(N, gens, base, lim, j, nu, t)

        for x in nu[j]:
            if x is not None:
                pw = x
                x.flag = False

def fillOrbits(N, gens, base, lim, nu):
    for i in range(len(base)):
        orbit = []  # partial orbit already found
        for y in nu[i]:
            if y is not None:
                j = y.permutation.array_form[base[i]]
                if j not in orbit:
                    orbit.append(j)
        for j in range(i + 1, len(base)):
            for x in nu[j]:
                if x is not None:
                    x1 = x.inverse()
                    orbit_x = [x.permutation.array_form[it] for it in orbit]
                    new_pts = [p for p in orbit_x if p not in orbit]

                    for p in new_pts:
                        t1 = x1 * (nu[i][x1.permutation.array_form[p]])
                        t1.words = simplify(t1.words)
                        if len(t1.words) < lim:
                            nu[i][p] = t1

In [7]:


# Options:
#   n: max number of rounds
#   s: reset each s rounds
#   w: limit for word size
#   so: sum  orbits size

#
def buildShortWordsSGS(N, gens, base, n, s, w, so):
    nu = [[None for _ in range(N)] for _ in range(len(base))]
    for i in range(len(base)):
        nu[i][base[i]] = PermWord(Permutation(N),[])
    nw = 0
    maximum = n
    lim = float(w)
    cnt = 0
    with tqdm(total= so) as pbar:   
        iter_gen = chain.from_iterable(product(list(gens), repeat=i) for i in range(1,12))
        for gen in iter_gen:
            cnt += 1
            if cnt >= maximum or nw == so :
                break

            perm = gen_perm_from_word(gens,gen)
            pw = PermWord(perm,list(gen))
            oneRound(N, gens, base, lim, 0, nu, pw)
            nw0 =nw
            nw =  np.sum([np.sum([x!=None for x in nu_i]) for nu_i in nu])
            deltanw = nw-nw0
            pbar.update(deltanw)
            if cnt % s == 0:
                oneImprove(N, gens, base, lim, nu)
                if nw < so:
                    fillOrbits(N, gens, base, lim, nu)
                lim *= 5 / 4
                
    return nu

def factorizeM(N, gens, base, nu, target):
    result_list = []
    perm = target
    for i in range(len(base)):
        omega = perm.array_form[base[i]]
        perm *= nu[i][omega].permutation
        result_list = result_list + nu[i][omega].words

    if not perm == Permutation(size = N+1):
        print("failed to reach identity, permutation not in group")

    return simplify(invwords(result_list))

def gen_perm_from_word(gens,words):
    res = gens[words[0]]
    for w in words[1:]:
        res = res * gens[w]
    return res

def invwords(ws):
    inv_ws = [geninvs[g] for g in ws]
    inv_ws.reverse() 
    return inv_ws


#remove invers generators in concatenation
def simplify(ff):
    if not ff:
        return ff
    # find adjacent inverse generators
    zero_sum_indices = [(i, i + 1) for i in range(len(ff) - 1) if ff[i] == geninvs[ff[i + 1]] ]
    # If there is no more simplications
    if not zero_sum_indices:
        return ff
    # remove inverse pairs
    for start, end in zero_sum_indices:
        return simplify(ff[:start] + ff[end + 1:])
    return ff

In [8]:
# Test Minwictz algorithm
    
def test_SGS(N,nu,base):
    result = True
    for i in range(len(base)):
        # diagonal identities
        p = nu[i][base[i]].words
        if p != []:
            result = False
            print('fail identity')
            
        for j in range(N):
            if j in nu[i]:
                p =nu[i][j].permutation.array_form 
                # stabilizes points upto i
                for k in range(i):
                    p2 = p[base[k]]
                    if p2 != base[k]:
                        result = False
                        print('fail stabilizer',i,j,k)

                
                # correct transversal at i
                if p[j] != base[i]:
                    result = False  
                    print('fail traversal ',i,j)
    return result

def quality(N, nu, base):
    result = 0
    for i in range(len(base)):
        maxlen = 0
        for j in range(N):
            if nu[i][j] is not None:
                wordlen = len(nu[i][j].words)
                if wordlen > maxlen:
                    maxlen = wordlen
        result += maxlen
    return result

In [9]:
def ReduceFactor(PG,s,maxl = 15):
    l0 = len(s)
    l1 = l0+1
    while (l0!=l1):
        a= []
        # Find shortcuts using fast factorization of permutations
        ls = len(s)
        for i in range(ls):
            for j in range(maxl):
                j1= i+j
                if j1 < ls:
                    target = applyPerm(s[i:j1+1])
                    sol = PG.FactorPermutation(target)
                    if len(sol)<(j1-i):
                        a.append((i,j1+1,{'weight':len(sol)}))
        # Find shortest path
        G = nx.path_graph(ls+1,nx.DiGraph)
        G.add_edges_from(a)
        opt = nx.dijkstra_path(G, 0, ls)
        #opt = nx.shortest_path(G, 0, ls, method = 'bellman-ford')
        sopt =[]
        for i,j in zip(opt,opt[1:]):
            if j-i == 1:
                sopt.append(s[i])
            else:
                target = applyPerm(s[i:j])
                sol= PG.FactorPermutation(target)
                sopt = sopt + sol
        l1=l0
        s = sopt
        l0 = len(s) 

        if applyPerm(s) != applyPerm(sopt):
            print('error',s,sopt)  
            break
    return sopt

The Schreier–Sims algorithm is an algorithm in computational group theory, named after the mathematicians Otto Schreier and Charles Sims. This algorithm can find the order of a finite permutation group, test membership (is a given permutation contained in a group?), and many other tasks in polynomial time. It was introduced by Sims in 1970, based on Schreier's subgroup lemma. The timing was subsequently improved by Donald Knuth in 1991. Later, an even faster randomized version of the algorithm was developed. Sympy has an implementation of both deterministic and randomized versions of the algorithm.

Let's try an example of the Schreier–Sims algorithm using the permutation group of the 2x2x2 Rubik's Cube.

Build the Permutation Group

In [10]:
c_type ='cube_2/2/2'
gens = eval(info[info.puzzle_type == c_type].allowed_moves.values[0])
N =max([max(gens[g]) for g in gens])+1
print('Number of generators:',len(gens),'Number of points:',N)
gens = {g:Permutation(gens[g],size = N) for g in gens}
PG = SGSPermutationGroup(gens)
print('Order:',PG.G.order())

Number of generators: 6 Number of points: 24
Order: 88179840


Obtain a Strong Generating Set (also called a base) using the Schreier–Sims algorithm.

This base allow to factor **any permutation** in the Permutation Group as a composition of **one permutations in each stabilizer** ( or the identity). The problem is that in general those permutations does'nt have a short expression in terms of the group generators .

The choice of this base is not unique, allowing to obtain different factorizations of the original permutation depending on yhe bas chosen.


In [11]:
PG.G.schreier_sims()
print('Stabilized points:',PG.G.base)
for g in PG.G.basic_stabilizers:
    print(g)

Stabilized points: [2, 0, 1, 6, 3, 7, 11]
PermutationGroup([
    (23)(2 19 21 8)(3 17 20 10)(4 6 7 5),
    (0 18 23 9)(1 16 22 11)(12 13 15 14),
    (1 5 21 14)(3 7 23 12)(8 10 11 9),
    (23)(0 4 20 15)(2 6 22 13)(16 17 19 18),
    (6 18 14 10)(7 19 15 11)(20 22 23 21),
    (23)(0 1 3 2)(4 16 12 8)(5 17 13 9),
    (3 7 5 21 8 10)(11 15 14 18 23 22),
    (3 11 7 18)(5 23 21 15)(8 14 10 22),
    (7 10 21)(11 14 23)(15 18 22),
    (11 23 14)(15 18 22),
    (7 14 10 23 21 11)(15 22 18),
    (11 15)(14 18)(22 23)])
PermutationGroup([
    (0 18 23 9)(1 16 22 11)(12 13 15 14),
    (1 5 21 14)(3 7 23 12)(8 10 11 9),
    (6 18 14 10)(7 19 15 11)(20 22 23 21),
    (3 7 5 21 8 10)(11 15 14 18 23 22),
    (3 11 7 18)(5 23 21 15)(8 14 10 22),
    (7 10 21)(11 14 23)(15 18 22),
    (11 23 14)(15 18 22),
    (7 14 10 23 21 11)(15 22 18),
    (11 15)(14 18)(22 23)])
PermutationGroup([
    (1 5 21 14)(3 7 23 12)(8 10 11 9),
    (6 18 14 10)(7 19 15 11)(20 22 23 21),
    (3 7 5 21 8 10)(11 15 14 18 23 

For instance for the firsts 2x2x2  cube the permutation is:

In [12]:
for i in range(3):
    sol = sub[sub.id ==i].moves.values[0]
    sol = sol.split('.')
    sol.reverse()
    target = applyPerm(sol)
    print(sol,target)
    seq = PG.G.coset_factor(target)
    display(seq)


['-f1', 'r1'] (0 9 23 16 1 11 13 12 14)(2 6 22 17 19 18 4 20 15)


[Permutation(23)(0, 4, 20, 15)(2, 6, 22, 13)(16, 17, 19, 18),
 Permutation(0, 9, 23, 18)(1, 11, 22, 16)(12, 14, 15, 13),
 Permutation(23),
 Permutation(23),
 Permutation(23),
 Permutation(23),
 Permutation(23)]

['-f0', '-f1', '-d1', '-d0', 'r0', '-d1', '-d0', '-f1', '-d0', 'd1', 'f1', '-r1', '-f1', '-d1', '-d0', 'r0', '-d1', 'f1', '-d0', '-d0', '-r0', 'd0', '-d1', 'f1', '-f0', 'f1', '-r1', '-r0', 'd0', '-f1', '-d0', 'r0', '-d0', '-d0', '-r0', '-d0', '-r0', 'd0', 'd1', 'f1', '-d0', '-d0', '-r0', 'd0', 'd1', 'f1', 'r0', 'd1', 'f1', '-r1', '-d1', 'f1', '-r0', '-f1', 'f0', '-r0', 'd0', '-f1', '-d0', '-f1', '-r0', 'd0', 'f1'] (0 16 13)(1 22 12 15 9 18)(2 7 14 19 5 4 10 23 6 3 17 21 11 20 8)


[Permutation(23)(0, 6, 22, 13, 19, 18, 16, 20, 15)(2, 7, 5, 4, 10, 3, 17, 21, 8),
 Permutation(0, 18, 23, 9)(1, 16, 22, 11)(12, 13, 15, 14),
 Permutation(1, 6, 14, 22, 21, 5)(3, 12, 20, 23, 15, 7)(8, 9, 19, 11, 18, 10),
 Permutation(6, 18, 14, 10)(7, 19, 15, 11)(20, 22, 23, 21),
 Permutation(3, 11, 7, 18)(5, 23, 21, 15)(8, 14, 10, 22),
 Permutation(7, 10, 21)(11, 14, 23)(15, 18, 22),
 Permutation(11, 22)(14, 15)(18, 23)]

['-f1', 'r1', 'f1', '-d0', 'r0', 'f1', 'r0', '-d1', 'f1', 'r0', 'f0', 'r1', '-f0', '-r1', '-f0', '-r0', 'f1', 'r1', 'f1', '-d0', 'r0', 'f1', 'r0', '-d1', '-f0', '-d1', '-f1', '-d0', '-f1', '-r1', '-f1', '-r1', '-r1', '-f0', '-f1', '-r1', '-f1', '-r1', '-r1', '-f0', '-r0', '-f1', '-r1', '-r0', '-f1', '-f0', '-r1', '-f0', '-r0', '-d0', 'r0', '-f0', '-r1', '-f0', '-r0', 'f1', '-f0', '-d1', 'r0', '-d1', 'd0', 'f1'] (0 10 2 16 21 4 13 7 17)(1 15 6 9 22 19 12 18 20)(3 5 8)(11 14 23)


[Permutation(23)(0, 20, 10, 3, 18, 17)(2, 16, 19, 21, 8, 22)(4, 13, 6, 7, 5, 15),
 Permutation(0, 20, 23)(1, 21, 22)(6, 14, 13)(7, 15, 12)(9, 10, 18)(11, 16, 19),
 Permutation(1, 5, 21, 14)(3, 7, 23, 12)(8, 10, 11, 9),
 Permutation(6, 18, 14, 10)(7, 19, 15, 11)(20, 22, 23, 21),
 Permutation(3, 22, 10, 14)(5, 18, 7, 11)(8, 15, 21, 23),
 Permutation(7, 23)(10, 11)(14, 21),
 Permutation(11, 22)(14, 15)(18, 23)]

In order to be able to write this factors in terms of the generators of the Permutation Group we need to mantain during the calculations the expresion for the permutations in terms of the generators, and try to minimimize the lenght of these expressions. This is what does the Minkwitz algorithm.



In [20]:
geninvs = PG.geninvs
PG.getShortWords(n=100000, s=1000, w=500)
print()
for i in range(len(PG.base)):
    print(i, [PG.nu[i][j].words for j in PG.bo[i] if PG.nu[i][j] != None])
    print()

print(PG.base)

100%|██████████| 105/105 [00:00<00:00, 729.51it/s]


0 [['-d1'], ['-d1', '-d1'], [], ['d1'], ['f0', '-r1'], ['f1', '-r0', '-f1', 'r0', 'f1', 'r0', '-f1', '-r0', '-r0', 'f1', '-r0', '-r0', '-f1', 'r0', '-d1', 'f1', 'd1', '-r1', '-r1'], ['-r1'], ['-d1', '-r1', 'd1', 'f1', 'r0', '-d0', '-r1', '-r1'], ['f0'], ['r0', 'r0', 'd0', '-r1'], ['f1', 'r0', '-f1', '-r1', '-r1'], ['d0', 'r0', 'f0', 'd1', '-f0', '-r0', '-d0', '-f1', '-r1', '-r1'], ['r0', 'r0', 'd0', '-f0'], ['r1'], ['f1', '-d1', 'd0', '-f0', 'd0', '-r1', '-d0', '-d0', 'r0', '-f1', '-r0', '-r0', '-r1', '-r1'], ['f1', 'r0', '-f1', 'r0', 'f1', 'r0', '-f1', '-r0', '-r0', 'd0', 'r0', '-d0', '-r0', 'f0'], ['-f1', '-f1', '-d0', '-d0', '-f1', '-f0'], ['r1', '-f0'], ['-d0', 'f0', '-r0', '-f0', 'r0', 'r0', 'd1', '-f0', '-d1', 'd0', 'r0', '-d0', '-r0', 'd1'], ['-f0'], ['r0', 'd1', '-d0', '-f0', '-d1', '-f0'], ['-f0', '-f0'], ['-r1', '-r1'], ['f0', 'r0', '-d1', '-f0', '-r0', 'f1', '-f0', '-d1', 'r0', 'd1', '-r1', '-r1']]

1 [[], ['r0', 'r0', '-d0', '-f1', '-f1'], ['-d0', 'f0', '-r0', '-f0', 'r0',

Once we have the base elements written in terms of the generators, we can obtain the factorization of the original permutation in terms of the base elements instantly.

In [14]:
for i in range(10):
    sol = sub[sub.id ==i].moves.values[0]
    sol = sol.split('.')
    sol.reverse()
    target = applyPerm(sol)
    ss = PG.FactorPermutation(target)
    ss.reverse
    display(i,'.'.join(ss))


0

'-f1.r1'

1

'-d0.f0.-r0.-f0.r0.r0.d1.-f0.-d1.d0.r0.-d0.-r0.r1.r1.-d1.-r1.-r1.-d0.-r0.d0.-r0.-d0.r0.r0.f1.-r0.-f1.-r0.f1.-r0.-f1.-d0.-r0.-d0.r0.d0.-r0.-d0.d1.f0.-d1.-r0.-r0.f0.r0.-f0.d0.f1.-f0.d1.-r1.-d1.r1.r1.d0.-r0.-f1.-d1.r1.d1'

2

...

'r0.f1.r0.-f1.-r0.-r0.d0.d0.f1.-r0.-f1.-d0.-d1.-f1.d1.-r0.f1.r0.r0.-f1.d1.f0.d0.-d1.-r0.-r0.-d0.r0.r0.f1.-r0.d0.d0.r1.-d0.f0.-d0.d1.f1.d0.-r0.-r0.r1.r1.-d1.-f1.d1.-r0.f1.r0.r0.-f1.r0.r0.f1.-r0.-f1.-r0.f1.r0.-f1'

8

'r0.f1.r0.-f1.-r0.-r0.d0.d0.f1.-r0.-f1.-d0.d1.f0.-d1.-r0.-f0.-d0.f0.d0.d0.f0.-r0.-f0.r0.-f1.-d1.r1.d1.-r0.f1.f1.d0.-r0.-r0.r1.r1.f1.-r0.-f1'

9

'd0.f1.r0.-f1.-d0.-d0.r0.r0.f1.-r0.-f1.-r0.f1.d0.r0.f0.-d1.-f0.-r0.-d0.f1.r0.-f1.d0.d0.r0.r0.d0.-r0.-d0.d1.f0.-d1.-r0.-r0.f0.r0.-f0.d0.-f1.-r0.f1.-r0.-f1.r0.-d1.f1.d1.f0.-d0.-r0.-r0'

We apply this method to the some puzzle.

In [14]:
ltypes = [
'cube_2/2/2',
'wreath_6/6',
'wreath_7/7',
'globe_6/4'
]



In [15]:
nretry = 1
for type in ltypes:
    print(type)
   
    ids = sub[path.puzzle_type == type].id.values
    gens = eval(info[info.puzzle_type == type].allowed_moves.values[0])
    N =max([max(gens[g]) for g in gens])+1
    gens = {g:Permutation(gens[g],size = N) for g in gens}
    
    
    for _ in range(nretry):
        sols = sub[path.puzzle_type == type].moves.values
        ll = sub.loc[path.puzzle_type == type,'moves'] .map(lambda x: len(x.split("."))).sum()
        print('Sum moves ', ll)
        PG = SGSPermutationGroup(gens)
        print('Dim:',PG.N,'Lenght base;',len(PG.base),'Sum Orbits:',PG.so)
        geninvs = PG.geninvs
        PG.getShortWords(n=100000, s=1000, w=500)
        print(PG.CheckQuality())
    

        for i,sol in enumerate(sols):
            sol = sol.split('.')
            sol.reverse()
            target = applyPerm(sol)
            ss = PG.FactorPermutation(target)
            if len(ss) > len(sol):
                ss = sol
            ss = ReduceFactor(PG,ss,10)
            if target != applyPerm(ss):
                print('error',applyPerm(sol),applyPerm(ss)) 
                continue
            ss.reverse()
    
            flag,_ =val_score(ids[i],'.'.join(ss))
            #if len(ss)<len(sol):
            print(ids[i],len(sol),'->',len(ss),flag)
            if (len(ss)<len(sol) and flag):
                sub.loc[sub.id == ids[i],'moves'] = '.'.join(ss)
        ll = sub.loc[path.puzzle_type == type,'moves'] .map(lambda x: len(x.split("."))).sum()
        print( 'Sum moves', ll)

cube_2/2/2
Sum moves  2275
Dim: 24 Lenght base; 7 Sum Orbits: 105


100%|██████████| 105/105 [00:00<00:00, 159.59it/s]


(True, 110)
0 2 -> 2 True
1 63 -> 49 True
2 62 -> 24 True
3 92 -> 62 True
4 70 -> 38 True
5 54 -> 32 True
6 68 -> 38 True
7 83 -> 53 True
8 98 -> 54 True
9 76 -> 72 True
10 66 -> 60 True
11 63 -> 49 True
12 72 -> 38 True
13 131 -> 61 True
14 96 -> 64 True
15 68 -> 38 True
16 63 -> 33 True
17 62 -> 52 True
18 89 -> 51 True
19 82 -> 74 True
20 112 -> 58 True
21 96 -> 66 True
22 63 -> 35 True
23 53 -> 27 True
24 99 -> 29 True
25 61 -> 53 True
26 93 -> 33 True
27 73 -> 51 True
28 83 -> 65 True
29 82 -> 44 True
Sum moves 1405
wreath_6/6
Sum moves  7888
Dim: 10 Lenght base; 9 Sum Orbits: 54


100%|██████████| 54/54 [00:00<00:00, 454.15it/s]


(True, 98)
284 339 -> 51 True
285 335 -> 77 True
286 484 -> 36 True
287 482 -> 72 True
288 489 -> 61 True
289 351 -> 65 True
290 356 -> 72 True
291 380 -> 70 True
292 378 -> 50 True
293 322 -> 76 True
294 341 -> 61 True
295 486 -> 78 True
296 319 -> 69 True
297 452 -> 62 True
298 393 -> 67 True
299 398 -> 32 True
300 327 -> 71 True
301 397 -> 43 True
302 447 -> 43 True
303 412 -> 42 True
Sum moves 1198
wreath_7/7
Sum moves  7488
Dim: 12 Lenght base; 10 Sum Orbits: 75


100%|██████████| 75/75 [00:00<00:00, 636.36it/s]

(True, 186)


304 422 -> 103 True
305 441 -> 99 True
306 428 -> 144 True
307 409 -> 83 True
308 470 -> 90 True
309 408 -> 71 True
310 421 -> 127 True
311 706 -> 109 True
312 556 -> 115 True
313 453 -> 69 True
314 752 -> 97 True
315 508 -> 68 True
316 501 -> 60 True
317 406 -> 34 True
318 607 -> 108 True
Sum moves 1377
globe_6/4
Sum moves  16090
Dim: 56 Lenght base; 46 Sum Orbits: 413


100%|██████████| 413/413 [00:14<00:00, 27.68it/s]


(True, 2110)
373 2401 -> 600 True
374 2906 -> 588 True
375 3805 -> 778 True
376 2712 -> 968 True
377 4266 -> 684 True
Sum moves 3618
